In [1]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam,SGD
from sklearn.utils import shuffle
import pickle
import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
seed = 124
InputPath = "data/pickle/"
X = pickle.load(open(InputPath+"X2_4d_masked.pickle","rb"))
y = pickle.load(open(InputPath+"y2_4d_masked.pickle","rb"))
X,y=shuffle(X,y, random_state=seed)

In [ ]:
weights = [0, 0, 0, 0, 0]
for i in y:
    weights[i[0]]= weights[i[0]]+1
weights = np.array(weights)
weights 

In [ ]:
# weights = weights / sum(weights)
# weights

In [ ]:
weights = [0, 300, 300, 300, 0]
newX = []
newY = []
for i, (a, b) in enumerate(zip(X, y)):
    if(weights[b[0]]>0):
        weights[b[0]]= weights[b[0]]-1
        newX.append(a)
        newY.append(b[0]-1)

In [ ]:
weights = [0, 0, 0, 0, 0]
for i in newY:
    weights[i]= weights[i]+1
weights = np.array(weights)
weights 

In [ ]:
# X = X/255.0
y = tf.keras.utils.to_categorical(newY).astype(int)
print(X.shape)
print(y.shape)
print(np.amax(X))

print(np.amin(X))

In [ ]:
X = newX
X = np.array(X).reshape(-1,80,80,4)
y = np.array(y).reshape(-1,3)
X,y=shuffle(X,y, random_state=28)

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=seed)

In [ ]:
# plt.figure(figsize=(50,50))
# for i in range(15):
#     plt.subplot(5,3,i+1)
#     plt.xticks([])
#     plt.yticks([])
#     plt.grid(False)
#     plt.imshow(train_X[i], cmap=plt.cm.binary)
#     plt.xlabel(str(train_y[i]))
# plt.show()

In [ ]:
# NAME = "Cnn64x2-{}-fix_X_withActivation".format(int(time.time()))
# from models import simple_cnn_model
# model = simple_cnn_model()
# model.compile(loss=tf.keras.losses.categorical_crossentropy,optimizer=tf.keras.optimizers.Adadelta(), metrics=['accuracy'])
# model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
# sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(optimizer=sgd, loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# gpu_option = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
# sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [ ]:
# weights = [301, 493, 531, 329, 628]
# weights = [286, 459, 512, 317, 600]
# total = sum(weights)
# weights[:] = [x / total for x in weights]
# print(weights)
classWeights = { i : weights[i] for i in range(0, len(weights) ) }
classWeights

In [ ]:
NAME = "VGG-updated_{}".format(int(time.time()))
tensorboard = tf.keras.callbacks.TensorBoard(log_dir='logs/{}'.format(NAME))

In [ ]:
from models import VGG_16
model = VGG_16()
adam = Adam(learning_rate=1e-4, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer=adam, loss=tf.keras.losses.categorical_crossentropy,metrics=['accuracy'])
model.summary()

In [ ]:
# datagen = tf.keras.preprocessing.image.ImageDataGenerator(
#         featurewise_center=False,  # set input mean to 0 over the dataset
#         samplewise_center=False,  # set each sample mean to 0
#         featurewise_std_normalization=False,  # divide inputs by std of the dataset
#         samplewise_std_normalization=False,  # divide each input by its std
#         zca_whitening=False,  # apply ZCA whitening
#         rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
#         width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
#         height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
#         horizontal_flip=True,  # randomly flip images
#         vertical_flip=False) 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
                            rotation_range=360,
                             width_shift_range=0.05,
                             height_shift_range=0.05,
                             shear_range=0.10,
                             zoom_range=0.3,
                             horizontal_flip=True, 
                             vertical_flip=True,
                             brightness_range=[0.8,1.0],
                             rescale=1./255.
                            )

In [ ]:
datagen.fit(train_X)

In [ ]:
# it = datagen.flow(train_X, train_y, batch_size=1)
# plt.figure(figsize=(15,15))
# plt.tight_layout()
# for i in range(36):
#     plt.subplot(6,6,i+1)
#     plt.axis('off')
#     batch = it.next()
#     image = batch[0]
#     label = batch[1][0]
#     print(np.min(image), np.max(image), np.mean(image), np.std(image))
#     plt.imshow(image[0,:,:,0])
#     plt.text(10,30,label, color='green', size=20)
# plt.show()

In [ ]:
import os
from datetime import datetime

# checkpoint_path = "models/"+NAME+"-{epoch:02d}-{val_acc:.2f}.hdf5"

checkpoint_path = "models/"+NAME+".hdf5"
checkpoint_dir = os.path.dirname(checkpoint_path)

# cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_acc', verbose=0, 
#                                 save_best_only=True, save_weights_only=False, mode='auto', period=1)
# checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_acc', verbose=0, save_best_only=False, mode='max')
# cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_acc', verbose=1,
#     save_best_only=True, save_weights_only=False,save_frequency=100)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
# datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "logs/scalars/" + NAME

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
# import sys
# oldStdout = sys.stdout
# file = open('trainLog.txt', 'w')
# sys.stdout = file
# class_weight=classWeights,callbacks=[cp_callback,tensorboard_callback]
# history = model.fit_generator(datagen.flow(train_X, train_y, batch_size=32), epochs=100, 
#                     validation_data=(test_X, test_y))

history = model.fit(x=train_X, y=train_y, batch_size=64, epochs=100, 
                    validation_data=(test_X, test_y))

In [ ]:
import sklearn
import numpy as np
import Plot
test_loss, test_acc = model.evaluate(test_X,  test_y, verbose=2)

Plot.plot_accuracy_curve(history)
y_pred = model.predict(test_X)
map_characters = {0:'Dead',1:'1',2:'2',3:'3',4:'Complex'}
print('\n', sklearn.metrics.classification_report(np.where(test_y > 0)[1], np.argmax(y_pred, axis=1), target_names=list(map_characters.values())), sep='')    
Y_pred_classes = np.argmax(y_pred,axis=1) 
Y_true = np.argmax(test_y,axis=1)
# plotKerasLearningCurve()
# plt.show()  
Plot.plot_learning_curve(history)
plt.show()
map_characters = {0:'Dead',1:'1',2:'2',3:'3',4:'Complex'}
confusion_mtx = sklearn.metrics.confusion_matrix(Y_true, Y_pred_classes) 
Plot.plot_confusion_matrix(confusion_mtx, classes = list(map_characters.values())) 
plt.show()


In [ ]:
# import os
# from datetime import datetime

# NAME = "VGG_withoutRegulation-{}".format(int(time.time()))


# # checkpoint_path = "models/"+NAME+"-{epoch:02d}-{val_acc:.2f}.hdf5"

# checkpoint_path = "models/"+NAME+".hdf5"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# # cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_acc', verbose=0, 
# #                                 save_best_only=True, save_weights_only=False, mode='auto', period=1)
# # checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_acc', verbose=0, save_best_only=False, mode='max')
# # cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_acc', verbose=1,
# #     save_best_only=True, save_weights_only=False,save_frequency=100)

# cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
# # datetime.now().strftime("%Y%m%d-%H%M%S")
# logdir = "logs/scalars/" + NAME

# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
# history = model.fit(x=train_X, y=train_y, batch_size=64, epochs=1000, 
#                     validation_data=(test_X, test_y),class_weight=classWeights,callbacks=[cp_callback,tensorboard_callback])